In [12]:
pip install requests

Note: you may need to restart the kernel to use updated packages.Collecting requests
     ---------------------------------------- 62.8/62.8 kB 3.3 MB/s eta 0:00:00
     ---------------------------------------- 96.0/96.0 kB 5.7 MB/s eta 0:00:00
     ---------------------------------------- 61.5/61.5 kB ? eta 0:00:00
     -------------------------------------- 140.6/140.6 kB 8.2 MB/s eta 0:00:00
     -------------------------------------- 155.3/155.3 kB 9.7 MB/s eta 0:00:00



  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
pip install pandas

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip



     --------------------------------------- 10.3/10.3 MB 13.9 MB/s eta 0:00:00
     ------------------------------------- 499.4/499.4 kB 15.8 MB/s eta 0:00:00
     --------------------------------------- 14.8/14.8 MB 14.2 MB/s eta 0:00:00


In [21]:
import requests
import json
import time
import pandas as pd
from pprint import pprint


# 1. 楽天レシピのレシピカテゴリ一覧を取得する

res = requests.get('https://app.rakuten.co.jp/services/api/Recipe/CategoryList/20170426?applicationId=1050436635711957152')

json_data = json.loads(res.text)

parent_dict = {} # mediumカテゴリの親カテゴリの辞書

df = pd.DataFrame(columns=['category1','category2','category3','categoryId','categoryName'])

for category in json_data['result']['large']:
    df = pd.concat([df, pd.DataFrame({'category1':category['categoryId'],'category2':"",'category3':"",'categoryId':category['categoryId'],'categoryName':category['categoryName']}, index=[0])], ignore_index=True)

for category in json_data['result']['medium']:
    df = pd.concat([df, pd.DataFrame({'category1':category['parentCategoryId'],'category2':category['categoryId'],'category3':"",'categoryId':str(category['parentCategoryId'])+"-"+str(category['categoryId']),'categoryName':category['categoryName']}, index=[0])], ignore_index=True)
    parent_dict[str(category['categoryId'])] = category['parentCategoryId']

for category in json_data['result']['small']:
    df = pd.concat([df,pd.DataFrame({'category1':parent_dict[category['parentCategoryId']],'category2':category['parentCategoryId'],'category3':category['categoryId'],'categoryId':parent_dict[category['parentCategoryId']]+"-"+str(category['parentCategoryId'])+"-"+str(category['categoryId']),'categoryName':category['categoryName']}, index=[0])], ignore_index=True)

# 2. キーワードからカテゴリを抽出する
df_keyword = df.query('categoryName.str.contains("魚")', engine='python')

pprint(df_keyword)

     category1 category2 category3   categoryId        categoryName
2           32                               32              定番の魚料理
13          11                               11                   魚
67          11       445                 11-445              明太子・魚卵
68          11       446                 11-446              その他の魚介
347         32       339                 32-339                  煮魚
351         32       343                 32-343                 焼き魚
357         32       349                 32-349             その他の魚料理
409         36       495                 36-495               簡単魚料理
635         11        78       523    11-78-523                  赤魚
647         11        78      2027   11-78-2027                 太刀魚
978         18       184       949   18-184-949              魚介のサラダ
1300        26       261       414   26-261-414            魚介類を使わない
1315        26       262      1078  26-262-1078           魚のおもてなし料理
1580        32       339      1277  32-339-1277 